# Machine Learning Test 1

### By Ciniso Msibi - may22/comp/1423x

In [2]:
import matplotlib.pyplot as plt
import lxml.html as lh
import pandas as pd
import numpy as np
import requests
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
popul = "https://www.worldometers.info/world-population/uganda-population/"
page_handler = requests.get(popul)
doc = lh.fromstring(page_handler.content)
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]

[1, 1, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]

In [4]:
tr_elements=doc.xpath('//tr')
col = []
i = 0
for t in tr_elements[2]:
    i+=1
    name = t.text_content()
    print('%d:%s' % (i,name))
    col.append((name,[]))

1:Year
2:Population
3:Yearly %  Change
4:Yearly Change
5:Migrants (net)
6:Median Age
7:Fertility Rate
8:Density (P/Km²)
9:Urban Pop %
10:Urban Population
11:Country's Share of World Pop
12:World Population
13:UgandaGlobal Rank


In [5]:
for j in range(3,len(tr_elements)):
    T = tr_elements[j]
    
    if len(T)!=13:
        break
        
    i = 0
    
    for t in T.iterchildren():
        data = t.text_content()
        
        col[i][1].append(data)
        i+=1

In [6]:
[len(C) for (title,C) in col]

[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]

In [7]:
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)
df.head()

,Year,Population,Yearly % Change,Yearly Change,Migrants (net),Median Age,Fertility Rate,Density (P/Km²),Urban Pop %,Urban Population,Country's Share of World Pop,World Population,UgandaGlobal Rank
0,2020,"45,741,007",3.32 %,"1,471,413","168,694",16.7,5.01,229,25.7 %,"11,775,012",0.59 %,"7,794,798,739",31
1,2019,"44,269,594",3.61 %,"1,540,558","168,694",16.1,5.62,222,25.2 %,"11,135,655",0.57 %,"7,713,468,100",32
2,2018,"42,729,036",3.80 %,"1,562,448","168,694",16.1,5.62,214,24.6 %,"10,525,083",0.56 %,"7,631,091,040",33
3,2017,"41,166,588",3.83 %,"1,517,422","168,694",16.1,5.62,206,24.2 %,"9,942,312",0.55 %,"7,547,858,925",34
4,2016,"39,649,166",3.72 %,"1,423,713","168,694",16.1,5.62,198,23.7 %,"9,386,309",0.53 %,"7,464,022,049",35


In [8]:
cdf = df[['Year','Population','Yearly Change','Migrants (net)','Urban Population','World Population']]
df1 = cdf.rename(columns = {'Yearly Change': 'Yearly_Change','Migrants (net)':'Migrants','Urban Population': 'Urban_Population','World Population':'World_Population'})
df1.head()

,Year,Population,Yearly_Change,Migrants,Urban_Population,World_Population
0,2020,"45,741,007","1,471,413","168,694","11,775,012","7,794,798,739"
1,2019,"44,269,594","1,540,558","168,694","11,135,655","7,713,468,100"
2,2018,"42,729,036","1,562,448","168,694","10,525,083","7,631,091,040"
3,2017,"41,166,588","1,517,422","168,694","9,942,312","7,547,858,925"
4,2016,"39,649,166","1,423,713","168,694","9,386,309","7,464,022,049"


In [9]:
df1 = df1.replace('\+', '', regex=True)
df1 = df1.replace(',', '', regex=True)
df1.tail()

,Year,Population,Yearly_Change,Migrants,Urban_Population,World_Population
14,1970,9405600,283907,10001,629441,3700437046
15,1965,7986066,243794,25000,441548,3339583597
16,1960,6767095,175660,0,299817,3034949748
17,1955,5888793,146121,0,208359,2773019936
18,Year,Population,Yearly Change,Migrants (net),Urban Population,World Population


In [10]:
df1 = df1[df1['Yearly_Change'] != 'Yearly Change']

In [11]:
df1["Population"] = pd.to_numeric(df1["Population"], errors='ignore')
df1["Year"] = pd.to_numeric(df1["Year"], errors='ignore')
df1["Yearly_Change"] = pd.to_numeric(df1["Yearly_Change"], errors='ignore')
df1["Migrants"] = pd.to_numeric(df1["Migrants"], errors='ignore')
df1["Urban_Population"] = pd.to_numeric(df1["Urban_Population"], errors='ignore')
df1["World_Population"] = pd.to_numeric(df1["World_Population"], errors='ignore')

In [12]:
df1.corr()

,Year,Population,Yearly_Change,Migrants,Urban_Population,World_Population
Year,1.000000,0.968480,0.941166,0.483523,0.924069,0.999202
Population,0.968480,1.000000,0.988750,0.608954,0.989812,0.975045
Yearly_Change,0.941166,0.988750,1.000000,0.702897,0.988875,0.948742
Migrants,0.483523,0.608954,0.702897,1.000000,0.666373,0.495621
Urban_Population,0.924069,0.989812,0.988875,0.666373,1.000000,0.933754
World_Population,0.999202,0.975045,0.948742,0.495621,0.933754,1.000000


In [13]:
df1.dtypes

Year                int64
Population          int64
Yearly_Change       int64
Migrants            int64
Urban_Population    int64
World_Population    int64
dtype: object

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### a. Simple linear regression model

In [15]:
lx = df1[['Year']]
ly = df1['Population']

X_train,X_test,y_train,y_test = train_test_split(lx,ly,test_size=0.2)

clf = LinearRegression()

In [16]:
#training the model
clf.fit(lx,ly)


LinearRegression()

In [17]:
ly1 = clf.predict(lx)

In [18]:
from sklearn.metrics import r2_score
r2_score(ly,ly1)

0.9379534935349279

### b. Multiple linear regression model

In [19]:
mx1 = df1.drop(['Year'],axis=1).values
my1 = df1['Year'].values

In [20]:
X_trainm,X_testm,y_trainm,y_testm = train_test_split(mx1,my1,test_size=0.2)

In [21]:
#training the model
clf.fit(X_trainm,y_trainm)

LinearRegression()

In [22]:
clf.predict(X_testm)

array([1991.53911801, 1995.92663785, 2017.06158265, 2009.38201638])

In [23]:
r2_score(y_testm,clf.predict(X_testm))

0.9894921720827876

### c. Order 2 polynomial Regression model

In [24]:
o2x = df1[['Year']]
o2y = df1['Population']

from sklearn.preprocessing import PolynomialFeatures
X_train,X_test,y_train,y_test = train_test_split(o2x,o2y,test_size=0.2)

In [25]:
poly2 = PolynomialFeatures(degree=2)

In [26]:
X_poly2 = poly2.fit_transform(X_train)

In [27]:
poly2.fit(X_poly2,y_train)

PolynomialFeatures()

In [28]:
regress = LinearRegression()

In [29]:
regress.fit(X_poly2,y_train)

LinearRegression()

In [30]:
y_pred2 = regress.predict(X_poly2)

In [31]:
r2_score(y_train,y_pred2)

0.9969425397209073

### d. Order 3 polynomial regression model

In [32]:
poly3 = PolynomialFeatures(degree=3)

In [33]:
X_poly3 = poly3.fit_transform(X_train)

In [34]:
poly3.fit(X_poly3,y_train)

PolynomialFeatures(degree=3)

In [35]:
regress3 = LinearRegression()

In [36]:
regress3.fit(X_poly3,y_train)

LinearRegression()

In [37]:
y_pred3 = regress3.predict(X_poly3)
#y_pred3 = regress3.predict(2028)

In [38]:
r2_score(y_train,y_pred3)

0.9996647048391695

### e. Exponential Regression Model

In [39]:
from scipy.optimize import curve_fit
xx = df1.Year   
xy = df1.Population

def exp_func(x,a,b):
    return a*b**x

popt, pcov = curve_fit(exp_func,xx,xy)
yr_predict = exp_func(xx,popt[0],popt[1])

In [40]:
r2_score(xy,yr_predict)

0.4381483708926488

## 2. Evaluation of models

#### Simple linear regression

In [41]:
r2_score(ly,ly1)

0.9379534935349279

#### Multiple linear regression

In [42]:
r2_score(y_testm,clf.predict(X_testm))

0.9894921720827876

#### Order 2 Polynomial regression

In [43]:
r2_score(y_train,y_pred2)

0.9969425397209073

#### Order 3 Polynomial regression

In [44]:
r2_score(y_train,y_pred3)

0.9996647048391695

#### Exponential regression

In [45]:
r2_score(xy,yr_predict)

0.4381483708926488

## 3. The most appropriate model. Hence predicting the Uganda population by 2028.

##### - The Order 3 polynomial regression model is the most appropriate to use with an r2_score of 99.97%

In [46]:
y_new = regress3.predict(poly3.fit_transform([[2028]]))
y_new

array([58435664.1784668])

#### - In 2028 the population is estimated to be 58 435 664